# VisitKorea Web Crawler

이 노트북은 `https://korean.visitkorea.or.kr` 사이트의 여행지 정보를 크롤링합니다.

## 주요 기능
1. 목록 페이지에서 상세 페이지 링크 추출
2. 상세 페이지에서 제목, 메타 정보(설명, 키워드), 이미지, 주소 추출
3. `time.sleep`을 이용한 랜덤 딜레이 적용 (1~5초)

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urljoin

## 상세 페이지 크롤링 함수

개별 여행지 상세 페이지에서 필요한 정보를 추출하는 함수입니다.
- `topTitle`
- `meta[name="description"]` content
- `meta[name="keywords"]` content
- `img src` (모든 이미지)
- 주소 (`<li>주소...<span>`)

In [2]:
def crawl_detail_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 1. topTitle 추출
        # id="topTitle" 또는 class="topTitle" 등을 시도합니다.
        top_title = "N/A"
        if soup.find(id="topTitle"):
            top_title = soup.find(id="topTitle").get_text(strip=True)
        elif soup.find(class_="topTitle"):
            top_title = soup.find(class_="topTitle").get_text(strip=True)
            
        # 2. meta name="description" content 추출
        meta_desc = "N/A"
        meta_desc_tag = soup.find('meta', attrs={'name': 'description'})
        if meta_desc_tag and 'content' in meta_desc_tag.attrs:
            meta_desc = meta_desc_tag['content']
            
        # 3. meta name="keywords" content 추출
        meta_keywords = "N/A"
        meta_keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
        if meta_keywords_tag and 'content' in meta_keywords_tag.attrs:
            meta_keywords = meta_keywords_tag['content']
            
        # 4. img src="url" 모든 이미지 추출
        images = []
        for img in soup.find_all('img'):
            src = img.get('src')
            if src:
                # 상대 경로인 경우 절대 경로로 변환
                full_src = urljoin(url, src)
                images.append(full_src)
                
        # 5. 주소 추출 (<li> 중 "주소" 텍스트 포함하는 요소의 <span>)
        address = "N/A"
        for li in soup.find_all('li'):
            # 텍스트에 '주소'가 포함되어 있는지 확인
            if "주소" in li.get_text():
                span = li.find('span')
                if span:
                    address = span.get_text(strip=True)
                break
                
        return {
            "url": url,
            "title": top_title,
            "description": meta_desc,
            "keywords": meta_keywords,
            "address": address,
            "images_count": len(images),
            "images": images[:5] # 편의상 첫 5개만 출력 리스트에 포함 (전체는 필요 시 images 사용)
        }

    except Exception as e:
        print(f"Error crawling {url}: {e}")
        return None

## 목록 페이지 및 실행 로직

목록 페이지에서 상세 페이지 링크를 수집하고, 각 링크에 대해 크롤링을 수행합니다.

In [3]:

# 1. 목록 페이지 설정
list_url = "https://korean.visitkorea.or.kr/main/area_list.do?type=Place"
detail_base_pattern = "ms_detail.do"

# 크롤링할 대상 URL 리스트
target_urls = []

# 예시 URL 추가 (리스트 페이지 크롤링 실패 시 실행 확인용)
example_url = "https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=01b32883-1770-499e-9e0d-576736a55fbe"
target_urls.append(example_url)

print(f"Searching for links in: {list_url}")
try:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    # 목록 페이지 요청
    resp = requests.get(list_url, headers=headers)
    resp.raise_for_status()
    list_soup = BeautifulSoup(resp.text, 'html.parser')
    
    # 링크 추출 시도
    found_links = 0
    for a in list_soup.find_all('a', href=True):
        href = a['href']
        if detail_base_pattern in href:
            full_url = urljoin(list_url, href)
            if full_url not in target_urls:
                target_urls.append(full_url)
                found_links += 1
                
    print(f"Found {found_links} additional links from the list page.")
    if found_links == 0:
        print("Note: No links found automatically. The page might be loading data via JavaScript/AJAX.")
        print("Proceeding with the example URL provided.")

except Exception as e:
    print(f"Error getting list page: {e}")
    print("Proceeding with the example URL provided.")

print(f"Total URLs to crawl: {len(target_urls)}")

# 2. 크롤링 실행 Loop
crawled_data = []

for i, url in enumerate(target_urls):
    print(f"[{i+1}/{len(target_urls)}] Crawling: {url}")
    
    data = crawl_detail_page(url)
    
    if data:
        crawled_data.append(data)
        print(f"  -> Title: {data['title']}")
        print(f"  -> Address: {data['address']}")
        print(f"  -> Description len: {len(data['description'])}")
    
    # 3. Time Sleep (1~5초 랜덤)
    if i < len(target_urls) - 1:
        sleep_time = random.uniform(1, 5)
        print(f"  -> Sleeping for {sleep_time:.2f} seconds...")
        time.sleep(sleep_time)

print("\nAll crawling finished.")

Searching for links in: https://korean.visitkorea.or.kr/main/area_list.do?type=Place
Found 0 additional links from the list page.
Note: No links found automatically. The page might be loading data via JavaScript/AJAX.
Proceeding with the example URL provided.
Total URLs to crawl: 1
[1/1] Crawling: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=01b32883-1770-499e-9e0d-576736a55fbe
  -> Title: 
  -> Address: 우편번호 찾기
  -> Description len: 249

All crawling finished.
